In [ ]:
# """
# NFL Big Data Bowl 2026 - KAGGLE INFERENCE
# 适配Kaggle评分系统的独立predict函数
# 保证predict函数不依赖前面cell的状态，可独立运行
# """

# import torch
# import torch.nn as nn
# import numpy as np
# import pandas as pd
# import polars as pl
# from pathlib import Path
# from tqdm.auto import tqdm
# import warnings
# import os
# import pickle
# import joblib

# from torch.utils.data import TensorDataset, DataLoader

# warnings.filterwarnings('ignore')

# # ============================================================================
# # CONFIG（全部放在函数内或全局常量，不依赖外部cell）
# # ============================================================================

# class Config:
#     DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")
    
#     SEED = 42
#     N_FOLDS = 5
#     BATCH_SIZE = 256
#     WINDOW_SIZE = 12
#     HIDDEN_DIM = 128
#     MAX_FUTURE_HORIZON = 94
#     USE_PLAYERS_INTERACTIONS = True
    
#     FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
#     FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
#     DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# def set_seed(seed=42):
#     import random
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)

# set_seed(Config.SEED)

In [ ]:

# # ============================================================================
# # FEATURE ENGINEERING MODULES
# # ============================================================================

# def height_to_feet(height_str):
#     """身高转换工具函数"""
#     try:
#         ft, inches = map(int, str(height_str).split('-'))
#         return ft + inches/12
#     except:
#         return 6.0


# def create_base_features(input_df):
#     """
#     基础特征工程：物理属性、运动学、角色等基础特征
#     """
#     df = input_df.copy()
    
#     # 基础物理特征
#     df['player_height_feet'] = df['player_height'].apply(height_to_feet)
    
#     # 运动学特征
#     dir_rad = np.deg2rad(df['dir'].fillna(0))
#     delta_t = 0.1
#     df['velocity_x'] = (df['s'] + 0.5 * df['a'] * delta_t) * np.sin(dir_rad)
#     df['velocity_y'] = (df['s'] + 0.5 * df['a'] * delta_t) * np.cos(dir_rad)
#     df['acceleration_x'] = df['a'] * np.sin(dir_rad)
#     df['acceleration_y'] = df['a'] * np.cos(dir_rad)
    
#     # 角色特征
#     df['is_offense'] = (df['player_side'] == 'Offense').astype(int)
#     df['is_defense'] = (df['player_side'] == 'Defense').astype(int)
#     df['is_receiver'] = (df['player_role'] == 'Targeted Receiver').astype(int)
#     df['is_coverage'] = (df['player_role'] == 'Defensive Coverage').astype(int)
#     df['is_passer'] = (df['player_role'] == 'Passer').astype(int)
    
#     # 物理量特征
#     mass_kg = df['player_weight'].fillna(200.0) / 2.20462
#     df['momentum_x'] = df['velocity_x'] * mass_kg
#     df['momentum_y'] = df['velocity_y'] * mass_kg
#     df['kinetic_energy'] = 0.5 * mass_kg * (df['s'] ** 2)
    
#     # 球相关特征
#     if 'ball_land_x' in df.columns:
#         ball_dx = df['ball_land_x'] - df['x']
#         ball_dy = df['ball_land_y'] - df['y']
#         df['distance_to_ball'] = np.sqrt(ball_dx**2 + ball_dy**2)
#         df['angle_to_ball'] = np.arctan2(ball_dy, ball_dx)
#         df['ball_direction_x'] = ball_dx / (df['distance_to_ball'] + 1e-6)
#         df['ball_direction_y'] = ball_dy / (df['distance_to_ball'] + 1e-6)
#         df['closing_speed'] = (
#             df['velocity_x'] * df['ball_direction_x'] +
#             df['velocity_y'] * df['ball_direction_y']
#         )
    
#     return df


# def create_lag_features(df, window_size=8):
#     """
#     时序特征：滞后特征、EMA、滑动窗口特征
#     """
#     df = df.copy()
    
#     # 按分组排序
#     df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
#     gcols = ['game_id', 'play_id', 'nfl_id']
    
#     # Lag Features
#     for lag in [1, 2, 3, 4, 5]:
#         df[f'x_lag{lag}'] = df.groupby(gcols)['x'].shift(lag)
#         df[f'y_lag{lag}'] = df.groupby(gcols)['y'].shift(lag)
#         df[f'velocity_x_lag{lag}'] = df.groupby(gcols)['velocity_x'].shift(lag)
#         df[f'velocity_y_lag{lag}'] = df.groupby(gcols)['velocity_y'].shift(lag)
    
#     # EMA Features
#     df['velocity_x_ema'] = df.groupby(gcols)['velocity_x'].transform(
#         lambda x: x.ewm(alpha=0.3, adjust=False).mean()
#     )
#     df['velocity_y_ema'] = df.groupby(gcols)['velocity_y'].transform(
#         lambda x: x.ewm(alpha=0.3, adjust=False).mean()
#     )
#     df['speed_ema'] = df.groupby(gcols)['s'].transform(
#         lambda x: x.ewm(alpha=0.3, adjust=False).mean()
#     )
    
#     # Rolling Features
#     df['velocity_x_roll'] = df.groupby(gcols)['velocity_x'].transform(
#         lambda x: x.rolling(window_size, min_periods=1).mean()
#     )
#     df['velocity_y_roll'] = df.groupby(gcols)['velocity_y'].transform(
#         lambda x: x.rolling(window_size, min_periods=1).mean()
#     )
    
#     return df


# def add_advanced_features(df):
#     """
#     高级特征：衍生特征、交互特征、战术特征
#     """
#     print("Adding advanced features...")
#     df = df.copy()
#     df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
#     gcols = ['game_id', 'play_id', 'nfl_id']

#     # 位置差分特征
#     if 'distance_to_ball' in df.columns:
#         df['distance_to_ball_change'] = df.groupby(gcols)['distance_to_ball'].diff().fillna(0)
#         df['distance_to_ball_accel'] = df.groupby(gcols)['distance_to_ball_change'].diff().fillna(0)
#         df['time_to_intercept'] = (df['distance_to_ball'] / 
#                                   (np.abs(df['distance_to_ball_change']) + 0.1)).clip(0, 10)

#     # 朝向球的特征
#     if 'ball_direction_x' in df.columns:
#         df['velocity_alignment'] = (
#             df['velocity_x'] * df['ball_direction_x'] +
#             df['velocity_y'] * df['ball_direction_y']
#         )
#         df['velocity_perpendicular'] = (
#             df['velocity_x'] * (-df['ball_direction_y']) +
#             df['velocity_y'] * df['ball_direction_x']
#         )
#         if 'acceleration_x' in df.columns:
#             df['accel_alignment'] = (
#                 df['acceleration_x'] * df['ball_direction_x'] +
#                 df['acceleration_y'] * df['ball_direction_y']
#             )

#     # 速度差分特征
#     if 'velocity_x' in df.columns:
#         df['velocity_x_change'] = df.groupby(gcols)['velocity_x'].diff().fillna(0)
#         df['velocity_y_change'] = df.groupby(gcols)['velocity_y'].diff().fillna(0)
#         df['speed_change'] = df.groupby(gcols)['s'].diff().fillna(0)
#         df['direction_change'] = df.groupby(gcols)['dir'].diff().fillna(0)
#         df['direction_change'] = df['direction_change'].apply(
#             lambda x: x if abs(x) < 180 else x - 360 * np.sign(x)
#         )

#     # 场地位置特征
#     df['dist_from_left'] = df['y']
#     df['dist_from_right'] = 53.3 - df['y']
#     df['dist_from_sideline'] = np.minimum(df['dist_from_left'], df['dist_from_right'])
#     df['dist_from_endzone'] = np.minimum(df['x'], 120 - df['x'])

#     # 角色特定特征
#     if 'is_receiver' in df.columns and 'velocity_alignment' in df.columns:
#         df['receiver_optimality'] = df['is_receiver'] * df['velocity_alignment']
#         df['receiver_deviation'] = df['is_receiver'] * np.abs(df.get('velocity_perpendicular', 0))
#     if 'is_coverage' in df.columns and 'closing_speed' in df.columns:
#         df['defender_closing_speed'] = df['is_coverage'] * df['closing_speed']

#     # 时间特征
#     df['frames_elapsed'] = df.groupby(gcols).cumcount()
#     df['normalized_time'] = df.groupby(gcols)['frames_elapsed'].transform(
#         lambda x: x / (x.max() + 1)
#     )

#     # 基础物理衍生特征
#     df['speed_squared'] = df['s'] ** 2
#     df['accel_magnitude'] = np.sqrt(df['acceleration_x'] ** 2 + df['acceleration_y'] ** 2)
#     df['orientation_diff'] = np.abs(df['o'] - df['dir'])
#     df['orientation_diff'] = np.minimum(df['orientation_diff'], 360 - df['orientation_diff'])

#     if 'nearest_opp_dist' in df.columns:
#         df['pressure'] = 1 / np.maximum(df['nearest_opp_dist'], 0.5)
#         df['under_pressure'] = (df['nearest_opp_dist'] < 3).astype(int)
#         df['pressure_x_speed'] = df['pressure'] * df['s']

#     if 'mirror_wr_vx' in df.columns:
#         s_safe = np.maximum(df['s'], 0.1)
#         df['mirror_similarity'] = (
#                 df['velocity_x'] * df['mirror_wr_vx'] +
#                 df['velocity_y'] * df['mirror_wr_vy']
#         ) / s_safe
#         df['mirror_offset_dist'] = np.sqrt(
#             df['mirror_offset_x'] ** 2 + df['mirror_offset_y'] ** 2
#         )
#         df['mirror_alignment'] = df['mirror_similarity'] * df['is_coverage']

#     return df


# def get_opponent_features(input_df):
#     """
#     对手交互特征：计算球员与对手之间的空间和运动关系
#     """
#     features = []
    
#     for (gid, pid), group in tqdm(input_df.groupby(['game_id', 'play_id']),
#                                 desc="🏈 Opponents", leave=False):
#         last = group.sort_values('frame_id').groupby('nfl_id').last()

#         if len(last) < 2:
#             continue

#         positions = last[['x', 'y']].values
#         sides = last['player_side'].values
#         speeds = last['s'].values
#         directions = last['dir'].values
#         roles = last['player_role'].values

#         receiver_mask = np.isin(roles, ['Targeted Receiver', 'Other Route Runner'])

#         for i, (nid, side, role) in enumerate(zip(last.index, sides, roles)):
#             opp_mask = sides != side

#             feat = {
#                 'game_id': gid, 'play_id': pid, 'nfl_id': nid,
#                 'nearest_opp_dist': 50.0, 'closing_speed': 0.0,
#                 'num_nearby_opp_3': 0, 'num_nearby_opp_5': 0,
#                 'mirror_wr_vx': 0.0, 'mirror_wr_vy': 0.0,
#                 'mirror_offset_x': 0.0, 'mirror_offset_y': 0.0,
#                 'mirror_wr_dist': 50.0,
#             }

#             if not opp_mask.any():
#                 features.append(feat)
#                 continue

#             opp_positions = positions[opp_mask]
#             distances = np.sqrt(((positions[i] - opp_positions) ** 2).sum(axis=1))

#             if len(distances) == 0:
#                 features.append(feat)
#                 continue

#             nearest_idx = distances.argmin()
#             feat['nearest_opp_dist'] = distances[nearest_idx]
#             feat['num_nearby_opp_3'] = (distances < 3.0).sum()
#             feat['num_nearby_opp_5'] = (distances < 5.0).sum()

#             # 计算相对速度
#             my_vx = speeds[i] * np.sin(np.deg2rad(directions[i]))
#             my_vy = speeds[i] * np.cos(np.deg2rad(directions[i]))
#             opp_speeds = speeds[opp_mask]
#             opp_dirs = directions[opp_mask]
#             opp_vx = opp_speeds[nearest_idx] * np.sin(np.deg2rad(opp_dirs[nearest_idx]))
#             opp_vy = opp_speeds[nearest_idx] * np.cos(np.deg2rad(opp_dirs[nearest_idx]))

#             rel_vx = my_vx - opp_vx
#             rel_vy = my_vy - opp_vy
#             to_me = positions[i] - opp_positions[nearest_idx]
#             to_me_norm = to_me / (np.linalg.norm(to_me) + 0.1)
#             feat['closing_speed'] = -(rel_vx * to_me_norm[0] + rel_vy * to_me_norm[1])

#             # 防守球员与接球手的镜像关系
#             if role == 'Defensive Coverage' and receiver_mask.any():
#                 rec_positions = positions[receiver_mask]
#                 rec_distances = np.sqrt(((positions[i] - rec_positions) ** 2).sum(axis=1))

#                 if len(rec_distances) > 0:
#                     closest_rec_idx = rec_distances.argmin()
#                     rec_indices = np.where(receiver_mask)[0]
#                     actual_rec_idx = rec_indices[closest_rec_idx]

#                     rec_vx = speeds[actual_rec_idx] * np.sin(np.deg2rad(directions[actual_rec_idx]))
#                     rec_vy = speeds[actual_rec_idx] * np.cos(np.deg2rad(directions[actual_rec_idx]))

#                     feat['mirror_wr_vx'] = rec_vx
#                     feat['mirror_wr_vy'] = rec_vy
#                     feat['mirror_wr_dist'] = rec_distances[closest_rec_idx]
#                     feat['mirror_offset_x'] = positions[i][0] - rec_positions[closest_rec_idx][0]
#                     feat['mirror_offset_y'] = positions[i][1] - rec_positions[closest_rec_idx][1]

#             features.append(feat)

#     return pd.DataFrame(features)

# def create_multi_window_rolling_features(df):
#     """多窗口滚动特征（3,5,10窗口的滚动平均和标准差）"""
#     new_cols = []
#     gcols = ['game_id', 'play_id', 'nfl_id']
    
#     for window in (3, 6, 9):
#         for col in ('velocity_x', 'velocity_y', 's', 'a'):
#             if col in df.columns:
#                 r_mean = df.groupby(gcols)[col].rolling(window, min_periods=1).mean()
#                 r_std = df.groupby(gcols)[col].rolling(window, min_periods=1).std()
#                 # 对齐索引
#                 r_mean = r_mean.reset_index(level=gcols, drop=True)
#                 r_std = r_std.reset_index(level=gcols, drop=True)
#                 df[f'{col}_roll{window}'] = r_mean
#                 df[f'{col}_std{window}'] = r_std.fillna(0.0)
#                 new_cols.extend([f'{col}_roll{window}', f'{col}_std{window}'])
#     return df, new_cols

# def create_qb_relative_features(df):
#     """四分卫相对特征（与四分卫的距离和方向关系）"""
#     need = ['x', 'y', 'velocity_x', 'velocity_y', 'dir', 'player_role', 'frame_id']
#     for c in need:
#         if c not in df.columns:
#             return df, []  # 缺列则安全跳过
    
#     out_cols = [
#         'qb_distance', 'vel_to_qb_alignment', 'vel_to_qb_perp',
#         'bearing_to_qb_signed', 'bearing_to_qb_sin', 'bearing_to_qb_cos'
#     ]
#     for c in out_cols:
#         if c not in df.columns:
#             df[c] = np.nan
    
#     key = ['game_id', 'play_id', 'frame_id']
#     for _, g in df.groupby(key, sort=False):
#         idx = g.index.values
        
#         # 找本帧 QB
#         qb_rows = g[g['player_role'] == 'Passer']
#         if qb_rows.empty:
#             continue
#         qb_x = float(qb_rows.iloc[0]['x'])
#         qb_y = float(qb_rows.iloc[0]['y'])
    
#         dx = g['x'].values.astype('float32') - qb_x
#         dy = g['y'].values.astype('float32') - qb_y
#         dist = np.sqrt(dx * dx + dy * dy) + 1e-6
#         ux, uy = dx / dist, dy / dist  # QB->player 单位向量
    
#         vx = g['velocity_x'].values.astype('float32')
#         vy = g['velocity_y'].values.astype('float32')
    
#         align = vx * ux + vy * uy
#         perp = vx * (-uy) + vy * ux
    
#         # bearing 差
#         dir_rad = np.deg2rad(g['dir'].fillna(0.0).astype('float32').values)
#         to_qb_angle = np.arctan2(-dy, -dx)  # player->QB
#         bearing = np.rad2deg(np.arctan2(np.sin(to_qb_angle - dir_rad), np.cos(to_qb_angle - dir_rad)))
    
#         df.loc[idx, 'qb_distance'] = dist
#         df.loc[idx, 'vel_to_qb_alignment'] = align
#         df.loc[idx, 'vel_to_qb_perp'] = perp
#         df.loc[idx, 'bearing_to_qb_signed'] = bearing
#         df.loc[idx, 'bearing_to_qb_sin'] = np.sin(np.deg2rad(bearing))
#         df.loc[idx, 'bearing_to_qb_cos'] = np.cos(np.deg2rad(bearing))
    
#     return df, out_cols

# def create_jerk_features(df):
#     """急动特征（加速度的变化率）"""
#     new_cols = []
#     gcols = ['game_id', 'play_id', 'nfl_id']
    
#     if 'a' in df.columns:
#         df['jerk'] = df.groupby(gcols)['a'].diff().fillna(0.0) * 0.1  # 假设delta_t=0.1
#         new_cols.append('jerk')
    
#     if {'acceleration_x', 'acceleration_y'}.issubset(df.columns):
#         df['jerk_x'] = df.groupby(gcols)['acceleration_x'].diff().fillna(0.0) * 0.1
#         df['jerk_y'] = df.groupby(gcols)['acceleration_y'].diff().fillna(0.0) * 0.1
#         new_cols.extend(['jerk_x', 'jerk_y'])
    
#     return df, new_cols

# def create_curvature_land_features(df):
#     """落点曲率特征（侧向偏差、角度差、曲率）"""
#     import numpy as np
#     new_cols = []
    
#     if {'ball_land_x', 'ball_land_y'}.issubset(df.columns):
#         # 侧向偏差和bearing_to_land
#         dx = df['ball_land_x'] - df['x']
#         dy = df['ball_land_y'] - df['y']
#         bearing = np.arctan2(dy, dx)
#         a_dir = np.deg2rad(df['dir'].fillna(0.0).values)
#         # 有符号方位差
#         df['bearing_to_land_signed'] = np.rad2deg(np.arctan2(np.sin(bearing - a_dir), np.cos(bearing - a_dir)))
#         # 侧向偏差
#         ux, uy = np.cos(a_dir), np.sin(a_dir)
#         df['land_lateral_offset'] = dy * ux - dx * uy  # >0 落点在左侧
    
#     # 曲率
#     ddir = df.groupby(['game_id', 'play_id', 'nfl_id'])['dir'].diff().fillna(0.0)
#     ddir = ((ddir + 180.0) % 360.0) - 180.0
#     curvature = np.deg2rad(ddir).astype('float32') / (df['s'].replace(0, np.nan).astype('float32') * 0.1 + 1e-6)
#     df['curvature_signed'] = curvature.fillna(0.0)
#     df['curvature_abs'] = df['curvature_signed'].abs()
    
#     # 窗口均值
#     for w in (3, 5):
#         if 'curvature_signed' in df.columns:
#             r = df.groupby(['game_id', 'play_id', 'nfl_id'])['curvature_signed'].rolling(w, min_periods=1).mean().reset_index(level=['game_id', 'play_id', 'nfl_id'], drop=True)
#             df[f'curv_signed_roll{w}'] = r
#         if 'curvature_abs' in df.columns:
#             r2 = df.groupby(['game_id', 'play_id', 'nfl_id'])['curvature_abs'].rolling(w, min_periods=1).mean().reset_index(level=['game_id', 'play_id', 'nfl_id'], drop=True)
#             df[f'curv_abs_roll{w}'] = r2
    
#     new_cols.extend([
#         'bearing_to_land_signed', 'land_lateral_offset',
#         'curvature_signed', 'curvature_abs',
#         'curv_signed_roll3', 'curv_abs_roll3',
#         'curv_signed_roll5', 'curv_abs_roll5'
#     ])
    
#     return df, [c for c in new_cols if c in df.columns]

# def get_feature_columns(df):
#     """
#     获取特征列列表（包含所有特征）
#     """
#     base_feature_cols = [
#         'x', 'y', 's', 'a', 'o', 'dir', 'frame_id',
#         'ball_land_x', 'ball_land_y',
#         'player_height_feet', 'player_weight',
#         'velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y',
#         'momentum_x', 'momentum_y', 'kinetic_energy',
#         'is_offense', 'is_defense', 'is_receiver', 'is_coverage', 'is_passer',
#         'distance_to_ball', 'angle_to_ball', 'ball_direction_x', 'ball_direction_y', 'closing_speed',
#         'x_lag1', 'y_lag1', 'velocity_x_lag1', 'velocity_y_lag1',
#         'x_lag2', 'y_lag2', 'velocity_x_lag2', 'velocity_y_lag2',
#         'x_lag3', 'y_lag3', 'velocity_x_lag3', 'velocity_y_lag3',
#         'x_lag4', 'y_lag4', 'velocity_x_lag4', 'velocity_y_lag4',
#         'x_lag5', 'y_lag5', 'velocity_x_lag5', 'velocity_y_lag5',
#         'velocity_x_ema', 'velocity_y_ema', 'speed_ema',
#         'velocity_x_roll', 'velocity_y_roll',

#         'distance_to_ball_change', 'distance_to_ball_accel', 'time_to_intercept',
#         'velocity_alignment', 'velocity_perpendicular', 'accel_alignment',
#         'velocity_x_change', 'velocity_y_change', 'speed_change', 'direction_change',
#         'dist_from_sideline', 'dist_from_endzone',
#         'receiver_optimality', 'receiver_deviation', 'defender_closing_speed',
#         'frames_elapsed', 'normalized_time', 'velocity_x_delta', 'velocity_y_delta',

#         # 添加对手交互特征
#         'nearest_opp_dist', 'closing_speed', 'num_nearby_opp_3', 'num_nearby_opp_5',
#         'mirror_wr_vx', 'mirror_wr_vy', 'mirror_offset_x', 'mirror_offset_y', 'mirror_wr_dist',
#         'pressure', 'under_pressure', 'pressure_x_speed',
#         'mirror_similarity', 'mirror_offset_dist', 'mirror_alignment'

#         # 'velocity_x_roll3', 'velocity_x_roll6', 'velocity_x_roll9', 'velocity_x_std3', 'velocity_x_std6', 'velocity_x_std9'
#         # 'velocity_y_roll3', 'velocity_y_roll6', 'velocity_y_roll9', 'velocity_y_std3', 'velocity_y_std6', 'velocity_y_std9',
#         # 's_roll3', 's_roll6', 's_roll9', 's_std3', 's_std6', 's_std9'
#         # 'a_roll3', 'a_roll6', 'a_roll9', 'a_std3', 'a_std6', 'a_std9'
#         # 'qb_distance', 'vel_to_qb_alignment', 'vel_to_qb_perp',
#         # 'bearing_to_qb_signed', 'bearing_to_qb_sin', 'bearing_to_qb_cos'
#                 # 急动特征
#         'jerk', 'jerk_x', 'jerk_y',
        
#         # 落点曲率特征
#         'bearing_to_land_signed', 'land_lateral_offset', 'curvature_signed', 'curvature_abs',
#         'curv_signed_roll3', 'curv_abs_roll3', 'curv_signed_roll5', 'curv_abs_roll5',
#     ]
    
#     # 合并所有特征列，只返回数据集中存在的
#     all_feature_cols = base_feature_cols
#     return [c for c in all_feature_cols if c in df.columns]


# # ============================================================================
# # MAIN SEQUENCE PREPARATION FUNCTION
# # ============================================================================

# def prepare_sequences_fixed(input_df, output_df=None, test_template=None, is_training=True, window_size=Config.WINDOW_SIZE):
#     """
#     准备序列数据的主函数（包含所有特征工程）
#     """
#     # 分步应用特征工程
#     input_df = create_base_features(input_df)
#     input_df = create_lag_features(input_df, window_size)
#     opponent_features = get_opponent_features(input_df)
#     input_df = input_df.merge(opponent_features, on=['game_id', 'play_id', 'nfl_id'], how='left')
#     input_df = add_advanced_features(input_df)
#     # input_df, _ = create_multi_window_rolling_features(input_df)
#     # input_df, _ = create_qb_relative_features(input_df)
#         # 急动特征
#     input_df, _ = create_jerk_features(input_df)
#     # 落点曲率特征
#     input_df, _ = create_curvature_land_features(input_df)

#     # 获取特征列
#     feature_cols = get_feature_columns(input_df)
#     print(f"使用 {len(feature_cols)} 个特征" + (" (训练)" if is_training else ""))
    
#     # 设置索引用于分组操作
#     input_df.set_index(['game_id', 'play_id', 'nfl_id'], inplace=True)
#     grouped = input_df.groupby(level=['game_id', 'play_id', 'nfl_id'])
    
#     if is_training:
#         target_rows = output_df
#         target_groups = target_rows[['game_id', 'play_id', 'nfl_id']].drop_duplicates()
#         sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids = [], [], [], [], []
        
#         for _, row in tqdm(target_groups.iterrows(), total=len(target_groups)):
#             key = (row['game_id'], row['play_id'], row['nfl_id'])
#             try:
#                 group_df = grouped.get_group(key)
#             except KeyError:
#                 continue
            
#             # 创建输入窗口
#             input_window = group_df.tail(window_size)
#             if len(input_window) < window_size:
#                 if is_training:
#                     continue
#                 pad_len = window_size - len(input_window)
#                 pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
#                 input_window = pd.concat([pad_df, input_window], ignore_index=True)
            
#             # 填充缺失值
#             input_window = input_window.fillna(group_df.mean(numeric_only=True))
#             seq = input_window[feature_cols].values
            
#             # 处理NaN值
#             if np.isnan(seq).any():
#                 if is_training:
#                     continue
#                 seq = np.nan_to_num(seq, nan=0.0)
            
#             sequences.append(seq)
            
#             # 获取对应的目标值
#             out_grp = target_rows[
#                 (target_rows['game_id'] == row['game_id']) &
#                 (target_rows['play_id'] == row['play_id']) &
#                 (target_rows['nfl_id'] == row['nfl_id'])
#             ].sort_values('frame_id')
            
#             last_x = input_window.iloc[-1]['x']
#             last_y = input_window.iloc[-1]['y']
#             dx = out_grp['x'].values - last_x
#             dy = out_grp['y'].values - last_y
#             targets_dx.append(dx)
#             targets_dy.append(dy)
#             targets_frame_ids.append(out_grp['frame_id'].values)
            
#             sequence_ids.append({
#                 'game_id': key[0],
#                 'play_id': key[1],
#                 'nfl_id': key[2],
#                 'frame_id': input_window.iloc[-1]['frame_id']
#             })
        
#         return sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids
    
#     else:
#         # 推理模式
#         target_groups = test_template[['game_id', 'play_id', 'nfl_id']].drop_duplicates()
#         sequences, sequence_ids = [], []
        
#         for _, row in target_groups.iterrows():
#             key = (row['game_id'], row['play_id'], row['nfl_id'])
#             try:
#                 group_df = grouped.get_group(key)
#             except KeyError:
#                 continue
            
#             # 创建输入窗口
#             input_window = group_df.tail(window_size)
#             if len(input_window) < window_size:
#                 pad_len = window_size - len(input_window)
#                 pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
#                 input_window = pd.concat([pad_df, input_window], ignore_index=True)
            
#             # 填充缺失值
#             input_window = input_window.fillna(group_df.mean(numeric_only=True))
#             seq = input_window[feature_cols].values
            
#             # 处理NaN值
#             if np.isnan(seq).any():
#                 seq = np.nan_to_num(seq, nan=0.0)
            
#             sequences.append(seq)
#             sequence_ids.append({
#                 'game_id': key[0],
#                 'play_id': key[1],
#                 'nfl_id': key[2],
#                 'frame_id': input_window.iloc[-1]['frame_id']
#             })
        
#         return sequences, sequence_ids

In [ ]:
# # ============================================================================
# # MODEL（与训练一致，定义在函数外但代码完整）
# # ============================================================================

# class ImprovedSeqModel(nn.Module):
#     def __init__(self, input_dim, horizon):
#         super().__init__()
#         self.horizon = horizon
#         self.gru = nn.GRU(input_dim, 128, num_layers=2, batch_first=True, dropout=0.1)
#         self.pool_ln = nn.LayerNorm(128)
#         self.pool_attn = nn.MultiheadAttention(128, num_heads=4, batch_first=True)
#         self.pool_query = nn.Parameter(torch.randn(1, 1, 128))
#         self.head = nn.Sequential(
#             nn.Linear(128, 128),
#             nn.GELU(),
#             nn.Dropout(0.2),
#             nn.Linear(128, horizon)
#         )
    
#     def forward(self, x):
#         h, _ = self.gru(x)
#         B = h.size(0)
#         q = self.pool_query.expand(B, -1, -1)
#         h_norm = self.pool_ln(h)
#         ctx, _ = self.pool_attn(q, h_norm, h_norm)
#         ctx = ctx.squeeze(1)
#         out = self.head(ctx)
#         out = torch.cumsum(out, dim=1)
#         return out

# # ============================================================================
# # INFERENCE 核心：predict函数（独立运行，不依赖外部状态）
# # ============================================================================

# def predict(test: pl.DataFrame, test_input: pl.DataFrame) -> pl.DataFrame:
#     # ===== 1. 准备数据 =====
#     test_pd = test.to_pandas()
#     test_input_pd = test_input.to_pandas()
    
#     # ===== 2. 特征工程（与训练完全一致）=====
#     sequences, sequence_ids = prepare_sequences_fixed(
#         test_input_pd, test_template=test_pd, is_training=False, window_size=Config.WINDOW_SIZE
#     )
    
#     if not sequences:
#         return pl.DataFrame({"x": [], "y": []})
    
#     X_test = np.array(sequences, dtype=object)
#     x_last = np.array([s[-1, 0] for s in X_test])
#     y_last = np.array([s[-1, 1] for s in X_test])
    
#     # ===== 3. 加载模型 & Scaler（首次运行时加载）=====
#     # 模型路径 —— 请根据你实际上传的模型路径修改！！！
#     MODEL_DIR = Path("/kaggle/input/1101addjerk/pytorch/default/1/1101addjerk")  # 必须上传模型到 input 目录，如 /kaggle/input/nfl-models/
    
#     model_x_paths = [
#         MODEL_DIR / f"model_x_fold{i+1}.pth" for i in range(Config.N_FOLDS)
#     ]
#     model_y_paths = [
#         MODEL_DIR / f"model_y_fold{i+1}.pth" for i in range(Config.N_FOLDS)
#     ]
#     scaler_paths = [
#         MODEL_DIR / f"scaler_fold{i+1}.pkl" for i in range(Config.N_FOLDS)
#     ]
    
#     # 检查路径是否存在，如果不存在则报错提示用户上传
#     for p in model_x_paths + model_y_paths + scaler_paths:
#         if not p.exists():
#             raise FileNotFoundError(f"请上传模型文件到 /kaggle/input/nfl-models/ ，缺失: {p.name}")

#     models_x = []
#     models_y = []
#     scalers = []
    
#     for i in range(Config.N_FOLDS):
#         # 模型X
#         model_x = ImprovedSeqModel(input_dim=X_test[0].shape[1], horizon=Config.MAX_FUTURE_HORIZON).to(Config.DEVICE)
#         model_x.load_state_dict(torch.load(model_x_paths[i], map_location=Config.DEVICE))
#         model_x.eval()
#         models_x.append(model_x)
        
#         # 模型Y
#         model_y = ImprovedSeqModel(input_dim=X_test[0].shape[1], horizon=Config.MAX_FUTURE_HORIZON).to(Config.DEVICE)
#         model_y.load_state_dict(torch.load(model_y_paths[i], map_location=Config.DEVICE))
#         model_y.eval()
#         models_y.append(model_y)
        
#         # Scaler
#         scaler = joblib.load(scaler_paths[i])
#         scalers.append(scaler)
    
#     # ===== 4. 预测 =====
#     all_dx, all_dy = [], []
#     for mx, my, sc in zip(models_x, models_y, scalers):
#         X_scaled = np.stack([sc.transform(s) for s in X_test])
#         X_tensor = torch.tensor(X_scaled.astype(np.float32)).to(Config.DEVICE)
        
#         mx.eval()
#         my.eval()
#         with torch.no_grad():
#             dx = mx(X_tensor).cpu().numpy()
#             dy = my(X_tensor).cpu().numpy()
#         all_dx.append(dx)
#         all_dy.append(dy)
    
#     ens_dx = np.mean(all_dx, axis=0)
#     ens_dy = np.mean(all_dy, axis=0)
    
#     # ===== 5. 构造提交结果 =====
#     rows = []
#     H = ens_dx.shape[1]  # 预测步数
    
#     for i, sid in enumerate(sequence_ids):
#         fids = test_pd[
#             (test_pd['game_id'] == sid['game_id']) &
#             (test_pd['play_id'] == sid['play_id']) &
#             (test_pd['nfl_id'] == sid['nfl_id'])
#         ]['frame_id'].sort_values().tolist()
        
#         for t, fid in enumerate(fids):
#             tt = min(t, H - 1)
#             px = np.clip(x_last[i] + ens_dx[i, tt], 0, 120)
#             py = np.clip(y_last[i] + ens_dy[i, tt], 0, 53.3)
#             rows.append({'x': px, 'y': py})
    
#     return pl.DataFrame(rows)

# # ============================================================================
# # Kaggle 推理服务器（自动调用predict）
# # ============================================================================

# # 确保导入官方评估库
# try:
#     from kaggle_evaluation.nfl_inference_server import NFLInferenceServer
# except ImportError as e:
#     raise ImportError("请确保使用 Kaggle 的 NFL Inference 评测内核")

# # 创建推理服务器并运行
# inference_server = NFLInferenceServer(predict)

# if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#     inference_server.serve()
# else:
#     inference_server.run_local_gateway(('/kaggle/input/nfl-big-data-bowl-2026-prediction/',))

In [ ]:
"""
NFL Big Data Bowl 2026 - KAGGLE INFERENCE
适配Kaggle评分系统的独立predict函数
保证predict函数不依赖前面cell的状态，可独立运行
"""

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
from tqdm.auto import tqdm
import warnings
import os
import pickle
import joblib

from torch.utils.data import TensorDataset, DataLoader

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIG（全部放在函数内或全局常量，不依赖外部cell）
# ============================================================================

class Config:
    DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")
    
    SEED = 42
    N_FOLDS = 5
    BATCH_SIZE = 256
    WINDOW_SIZE = 12
    HIDDEN_DIM = 128
    MAX_FUTURE_HORIZON = 94
    USE_PLAYERS_INTERACTIONS = True
    
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(Config.SEED)

In [ ]:

# ============================================================================
# FEATURE ENGINEERING MODULES
# ============================================================================

def height_to_feet(height_str):
    """身高转换工具函数"""
    try:
        ft, inches = map(int, str(height_str).split('-'))
        return ft + inches/12
    except:
        return 6.0


def create_base_features(input_df):
    """
    基础特征工程：物理属性、运动学、角色等基础特征
    """
    df = input_df.copy()
    
    # 基础物理特征
    df['player_height_feet'] = df['player_height'].apply(height_to_feet)
    
    # 运动学特征
    dir_rad = np.deg2rad(df['dir'].fillna(0))
    delta_t = 0.1
    df['velocity_x'] = (df['s'] + 0.5 * df['a'] * delta_t) * np.sin(dir_rad)
    df['velocity_y'] = (df['s'] + 0.5 * df['a'] * delta_t) * np.cos(dir_rad)
    df['acceleration_x'] = df['a'] * np.sin(dir_rad)
    df['acceleration_y'] = df['a'] * np.cos(dir_rad)
    
    # 角色特征
    df['is_offense'] = (df['player_side'] == 'Offense').astype(int)
    df['is_defense'] = (df['player_side'] == 'Defense').astype(int)
    df['is_receiver'] = (df['player_role'] == 'Targeted Receiver').astype(int)
    df['is_coverage'] = (df['player_role'] == 'Defensive Coverage').astype(int)
    df['is_passer'] = (df['player_role'] == 'Passer').astype(int)
    
    # 物理量特征
    mass_kg = df['player_weight'].fillna(200.0) / 2.20462
    df['momentum_x'] = df['velocity_x'] * mass_kg
    df['momentum_y'] = df['velocity_y'] * mass_kg
    df['kinetic_energy'] = 0.5 * mass_kg * (df['s'] ** 2)
    
    # 球相关特征
    if 'ball_land_x' in df.columns:
        ball_dx = df['ball_land_x'] - df['x']
        ball_dy = df['ball_land_y'] - df['y']
        df['distance_to_ball'] = np.sqrt(ball_dx**2 + ball_dy**2)
        df['angle_to_ball'] = np.arctan2(ball_dy, ball_dx)
        df['ball_direction_x'] = ball_dx / (df['distance_to_ball'] + 1e-6)
        df['ball_direction_y'] = ball_dy / (df['distance_to_ball'] + 1e-6)
        df['closing_speed'] = (
            df['velocity_x'] * df['ball_direction_x'] +
            df['velocity_y'] * df['ball_direction_y']
        )
    
    return df


def create_lag_features(df, window_size=8):
    """
    时序特征：滞后特征、EMA、滑动窗口特征
    """
    df = df.copy()
    
    # 按分组排序
    df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    gcols = ['game_id', 'play_id', 'nfl_id']
    
    # Lag Features
    for lag in [1, 2, 3, 4, 5]:
        df[f'x_lag{lag}'] = df.groupby(gcols)['x'].shift(lag)
        df[f'y_lag{lag}'] = df.groupby(gcols)['y'].shift(lag)
        df[f'velocity_x_lag{lag}'] = df.groupby(gcols)['velocity_x'].shift(lag)
        df[f'velocity_y_lag{lag}'] = df.groupby(gcols)['velocity_y'].shift(lag)
    
    # EMA Features
    df['velocity_x_ema'] = df.groupby(gcols)['velocity_x'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    df['velocity_y_ema'] = df.groupby(gcols)['velocity_y'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    df['speed_ema'] = df.groupby(gcols)['s'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    
    # Rolling Features
    df['velocity_x_roll'] = df.groupby(gcols)['velocity_x'].transform(
        lambda x: x.rolling(window_size, min_periods=1).mean()
    )
    df['velocity_y_roll'] = df.groupby(gcols)['velocity_y'].transform(
        lambda x: x.rolling(window_size, min_periods=1).mean()
    )
    
    return df


def add_advanced_features(df):
    """
    高级特征：衍生特征、交互特征、战术特征
    """
    print("Adding advanced features...")
    df = df.copy()
    df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    gcols = ['game_id', 'play_id', 'nfl_id']

    # 位置差分特征
    if 'distance_to_ball' in df.columns:
        df['distance_to_ball_change'] = df.groupby(gcols)['distance_to_ball'].diff().fillna(0)
        df['distance_to_ball_accel'] = df.groupby(gcols)['distance_to_ball_change'].diff().fillna(0)
        df['time_to_intercept'] = (df['distance_to_ball'] / 
                                  (np.abs(df['distance_to_ball_change']) + 0.1)).clip(0, 10)

    # 朝向球的特征
    if 'ball_direction_x' in df.columns:
        df['velocity_alignment'] = (
            df['velocity_x'] * df['ball_direction_x'] +
            df['velocity_y'] * df['ball_direction_y']
        )
        df['velocity_perpendicular'] = (
            df['velocity_x'] * (-df['ball_direction_y']) +
            df['velocity_y'] * df['ball_direction_x']
        )
        if 'acceleration_x' in df.columns:
            df['accel_alignment'] = (
                df['acceleration_x'] * df['ball_direction_x'] +
                df['acceleration_y'] * df['ball_direction_y']
            )

    # 速度差分特征
    if 'velocity_x' in df.columns:
        df['velocity_x_change'] = df.groupby(gcols)['velocity_x'].diff().fillna(0)
        df['velocity_y_change'] = df.groupby(gcols)['velocity_y'].diff().fillna(0)
        df['speed_change'] = df.groupby(gcols)['s'].diff().fillna(0)
        df['direction_change'] = df.groupby(gcols)['dir'].diff().fillna(0)
        df['direction_change'] = df['direction_change'].apply(
            lambda x: x if abs(x) < 180 else x - 360 * np.sign(x)
        )

    # 场地位置特征
    df['dist_from_left'] = df['y']
    df['dist_from_right'] = 53.3 - df['y']
    df['dist_from_sideline'] = np.minimum(df['dist_from_left'], df['dist_from_right'])
    df['dist_from_endzone'] = np.minimum(df['x'], 120 - df['x'])

    # 角色特定特征
    if 'is_receiver' in df.columns and 'velocity_alignment' in df.columns:
        df['receiver_optimality'] = df['is_receiver'] * df['velocity_alignment']
        df['receiver_deviation'] = df['is_receiver'] * np.abs(df.get('velocity_perpendicular', 0))
    if 'is_coverage' in df.columns and 'closing_speed' in df.columns:
        df['defender_closing_speed'] = df['is_coverage'] * df['closing_speed']

    # 时间特征
    df['frames_elapsed'] = df.groupby(gcols).cumcount()
    df['normalized_time'] = df.groupby(gcols)['frames_elapsed'].transform(
        lambda x: x / (x.max() + 1)
    )

    # 基础物理衍生特征
    df['speed_squared'] = df['s'] ** 2
    df['accel_magnitude'] = np.sqrt(df['acceleration_x'] ** 2 + df['acceleration_y'] ** 2)
    df['orientation_diff'] = np.abs(df['o'] - df['dir'])
    df['orientation_diff'] = np.minimum(df['orientation_diff'], 360 - df['orientation_diff'])

    if 'nearest_opp_dist' in df.columns:
        df['pressure'] = 1 / np.maximum(df['nearest_opp_dist'], 0.5)
        df['under_pressure'] = (df['nearest_opp_dist'] < 3).astype(int)
        df['pressure_x_speed'] = df['pressure'] * df['s']

    if 'mirror_wr_vx' in df.columns:
        s_safe = np.maximum(df['s'], 0.1)
        df['mirror_similarity'] = (
                df['velocity_x'] * df['mirror_wr_vx'] +
                df['velocity_y'] * df['mirror_wr_vy']
        ) / s_safe
        df['mirror_offset_dist'] = np.sqrt(
            df['mirror_offset_x'] ** 2 + df['mirror_offset_y'] ** 2
        )
        df['mirror_alignment'] = df['mirror_similarity'] * df['is_coverage']

    return df


def get_opponent_features(input_df):
    """
    对手交互特征：计算球员与对手之间的空间和运动关系
    """
    features = []
    
    for (gid, pid), group in tqdm(input_df.groupby(['game_id', 'play_id']),
                                desc="🏈 Opponents", leave=False):
        last = group.sort_values('frame_id').groupby('nfl_id').last()

        if len(last) < 2:
            continue

        positions = last[['x', 'y']].values
        sides = last['player_side'].values
        speeds = last['s'].values
        directions = last['dir'].values
        roles = last['player_role'].values

        receiver_mask = np.isin(roles, ['Targeted Receiver', 'Other Route Runner'])

        for i, (nid, side, role) in enumerate(zip(last.index, sides, roles)):
            opp_mask = sides != side

            feat = {
                'game_id': gid, 'play_id': pid, 'nfl_id': nid,
                'nearest_opp_dist': 50.0, 'closing_speed': 0.0,
                'num_nearby_opp_3': 0, 'num_nearby_opp_5': 0,
                'mirror_wr_vx': 0.0, 'mirror_wr_vy': 0.0,
                'mirror_offset_x': 0.0, 'mirror_offset_y': 0.0,
                'mirror_wr_dist': 50.0,
            }

            if not opp_mask.any():
                features.append(feat)
                continue

            opp_positions = positions[opp_mask]
            distances = np.sqrt(((positions[i] - opp_positions) ** 2).sum(axis=1))

            if len(distances) == 0:
                features.append(feat)
                continue

            nearest_idx = distances.argmin()
            feat['nearest_opp_dist'] = distances[nearest_idx]
            feat['num_nearby_opp_3'] = (distances < 3.0).sum()
            feat['num_nearby_opp_5'] = (distances < 5.0).sum()

            # 计算相对速度
            my_vx = speeds[i] * np.sin(np.deg2rad(directions[i]))
            my_vy = speeds[i] * np.cos(np.deg2rad(directions[i]))
            opp_speeds = speeds[opp_mask]
            opp_dirs = directions[opp_mask]
            opp_vx = opp_speeds[nearest_idx] * np.sin(np.deg2rad(opp_dirs[nearest_idx]))
            opp_vy = opp_speeds[nearest_idx] * np.cos(np.deg2rad(opp_dirs[nearest_idx]))

            rel_vx = my_vx - opp_vx
            rel_vy = my_vy - opp_vy
            to_me = positions[i] - opp_positions[nearest_idx]
            to_me_norm = to_me / (np.linalg.norm(to_me) + 0.1)
            feat['closing_speed'] = -(rel_vx * to_me_norm[0] + rel_vy * to_me_norm[1])

            # 防守球员与接球手的镜像关系
            if role == 'Defensive Coverage' and receiver_mask.any():
                rec_positions = positions[receiver_mask]
                rec_distances = np.sqrt(((positions[i] - rec_positions) ** 2).sum(axis=1))

                if len(rec_distances) > 0:
                    closest_rec_idx = rec_distances.argmin()
                    rec_indices = np.where(receiver_mask)[0]
                    actual_rec_idx = rec_indices[closest_rec_idx]

                    rec_vx = speeds[actual_rec_idx] * np.sin(np.deg2rad(directions[actual_rec_idx]))
                    rec_vy = speeds[actual_rec_idx] * np.cos(np.deg2rad(directions[actual_rec_idx]))

                    feat['mirror_wr_vx'] = rec_vx
                    feat['mirror_wr_vy'] = rec_vy
                    feat['mirror_wr_dist'] = rec_distances[closest_rec_idx]
                    feat['mirror_offset_x'] = positions[i][0] - rec_positions[closest_rec_idx][0]
                    feat['mirror_offset_y'] = positions[i][1] - rec_positions[closest_rec_idx][1]

            features.append(feat)

    return pd.DataFrame(features)

def create_multi_window_rolling_features(df):
    """多窗口滚动特征（3,5,10窗口的滚动平均和标准差）"""
    new_cols = []
    gcols = ['game_id', 'play_id', 'nfl_id']
    
    for window in (3, 6, 9):
        for col in ('velocity_x', 'velocity_y', 's', 'a'):
            if col in df.columns:
                r_mean = df.groupby(gcols)[col].rolling(window, min_periods=1).mean()
                r_std = df.groupby(gcols)[col].rolling(window, min_periods=1).std()
                # 对齐索引
                r_mean = r_mean.reset_index(level=gcols, drop=True)
                r_std = r_std.reset_index(level=gcols, drop=True)
                df[f'{col}_roll{window}'] = r_mean
                df[f'{col}_std{window}'] = r_std.fillna(0.0)
                new_cols.extend([f'{col}_roll{window}', f'{col}_std{window}'])
    return df, new_cols

def create_qb_relative_features(df):
    """四分卫相对特征（与四分卫的距离和方向关系）"""
    need = ['x', 'y', 'velocity_x', 'velocity_y', 'dir', 'player_role', 'frame_id']
    for c in need:
        if c not in df.columns:
            return df, []  # 缺列则安全跳过
    
    out_cols = [
        'qb_distance', 'vel_to_qb_alignment', 'vel_to_qb_perp',
        'bearing_to_qb_signed', 'bearing_to_qb_sin', 'bearing_to_qb_cos'
    ]
    for c in out_cols:
        if c not in df.columns:
            df[c] = np.nan
    
    key = ['game_id', 'play_id', 'frame_id']
    for _, g in df.groupby(key, sort=False):
        idx = g.index.values
        
        # 找本帧 QB
        qb_rows = g[g['player_role'] == 'Passer']
        if qb_rows.empty:
            continue
        qb_x = float(qb_rows.iloc[0]['x'])
        qb_y = float(qb_rows.iloc[0]['y'])
    
        dx = g['x'].values.astype('float32') - qb_x
        dy = g['y'].values.astype('float32') - qb_y
        dist = np.sqrt(dx * dx + dy * dy) + 1e-6
        ux, uy = dx / dist, dy / dist  # QB->player 单位向量
    
        vx = g['velocity_x'].values.astype('float32')
        vy = g['velocity_y'].values.astype('float32')
    
        align = vx * ux + vy * uy
        perp = vx * (-uy) + vy * ux
    
        # bearing 差
        dir_rad = np.deg2rad(g['dir'].fillna(0.0).astype('float32').values)
        to_qb_angle = np.arctan2(-dy, -dx)  # player->QB
        bearing = np.rad2deg(np.arctan2(np.sin(to_qb_angle - dir_rad), np.cos(to_qb_angle - dir_rad)))
    
        df.loc[idx, 'qb_distance'] = dist
        df.loc[idx, 'vel_to_qb_alignment'] = align
        df.loc[idx, 'vel_to_qb_perp'] = perp
        df.loc[idx, 'bearing_to_qb_signed'] = bearing
        df.loc[idx, 'bearing_to_qb_sin'] = np.sin(np.deg2rad(bearing))
        df.loc[idx, 'bearing_to_qb_cos'] = np.cos(np.deg2rad(bearing))
    
    return df, out_cols

def create_jerk_features(df):
    """急动特征（加速度的变化率）"""
    new_cols = []
    gcols = ['game_id', 'play_id', 'nfl_id']
    
    if 'a' in df.columns:
        df['jerk'] = df.groupby(gcols)['a'].diff().fillna(0.0) * 0.1  # 假设delta_t=0.1
        new_cols.append('jerk')
    
    if {'acceleration_x', 'acceleration_y'}.issubset(df.columns):
        df['jerk_x'] = df.groupby(gcols)['acceleration_x'].diff().fillna(0.0) * 0.1
        df['jerk_y'] = df.groupby(gcols)['acceleration_y'].diff().fillna(0.0) * 0.1
        new_cols.extend(['jerk_x', 'jerk_y'])
    
    return df, new_cols

def create_curvature_land_features(df):
    """落点曲率特征（侧向偏差、角度差、曲率）"""
    import numpy as np
    new_cols = []
    
    if {'ball_land_x', 'ball_land_y'}.issubset(df.columns):
        # 侧向偏差和bearing_to_land
        dx = df['ball_land_x'] - df['x']
        dy = df['ball_land_y'] - df['y']
        bearing = np.arctan2(dy, dx)
        a_dir = np.deg2rad(df['dir'].fillna(0.0).values)
        # 有符号方位差
        df['bearing_to_land_signed'] = np.rad2deg(np.arctan2(np.sin(bearing - a_dir), np.cos(bearing - a_dir)))
        # 侧向偏差
        ux, uy = np.cos(a_dir), np.sin(a_dir)
        df['land_lateral_offset'] = dy * ux - dx * uy  # >0 落点在左侧
    
    # 曲率
    ddir = df.groupby(['game_id', 'play_id', 'nfl_id'])['dir'].diff().fillna(0.0)
    ddir = ((ddir + 180.0) % 360.0) - 180.0
    curvature = np.deg2rad(ddir).astype('float32') / (df['s'].replace(0, np.nan).astype('float32') * 0.1 + 1e-6)
    df['curvature_signed'] = curvature.fillna(0.0)
    df['curvature_abs'] = df['curvature_signed'].abs()
    
    # 窗口均值
    for w in (3, 5):
        if 'curvature_signed' in df.columns:
            r = df.groupby(['game_id', 'play_id', 'nfl_id'])['curvature_signed'].rolling(w, min_periods=1).mean().reset_index(level=['game_id', 'play_id', 'nfl_id'], drop=True)
            df[f'curv_signed_roll{w}'] = r
        if 'curvature_abs' in df.columns:
            r2 = df.groupby(['game_id', 'play_id', 'nfl_id'])['curvature_abs'].rolling(w, min_periods=1).mean().reset_index(level=['game_id', 'play_id', 'nfl_id'], drop=True)
            df[f'curv_abs_roll{w}'] = r2
    
    new_cols.extend([
        'bearing_to_land_signed', 'land_lateral_offset',
        'curvature_signed', 'curvature_abs',
        'curv_signed_roll3', 'curv_abs_roll3',
        'curv_signed_roll5', 'curv_abs_roll5'
    ])
    
    return df, [c for c in new_cols if c in df.columns]

def get_feature_columns(df):
    """
    获取特征列列表（包含所有特征）
    """
    base_feature_cols = [
        'x', 'y', 's', 'a', 'o', 'dir', 'frame_id',
        'ball_land_x', 'ball_land_y',
        'player_height_feet', 'player_weight',
        'velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y',
        'momentum_x', 'momentum_y', 'kinetic_energy',
        'is_offense', 'is_defense', 'is_receiver', 'is_coverage', 'is_passer',
        'distance_to_ball', 'angle_to_ball', 'ball_direction_x', 'ball_direction_y', 'closing_speed',
        'x_lag1', 'y_lag1', 'velocity_x_lag1', 'velocity_y_lag1',
        'x_lag2', 'y_lag2', 'velocity_x_lag2', 'velocity_y_lag2',
        'x_lag3', 'y_lag3', 'velocity_x_lag3', 'velocity_y_lag3',
        'x_lag4', 'y_lag4', 'velocity_x_lag4', 'velocity_y_lag4',
        'x_lag5', 'y_lag5', 'velocity_x_lag5', 'velocity_y_lag5',
        'velocity_x_ema', 'velocity_y_ema', 'speed_ema',
        'velocity_x_roll', 'velocity_y_roll',

        'distance_to_ball_change', 'distance_to_ball_accel', 'time_to_intercept',
        'velocity_alignment', 'velocity_perpendicular', 'accel_alignment',
        'velocity_x_change', 'velocity_y_change', 'speed_change', 'direction_change',
        'dist_from_sideline', 'dist_from_endzone',
        'receiver_optimality', 'receiver_deviation', 'defender_closing_speed',
        'frames_elapsed', 'normalized_time', 'velocity_x_delta', 'velocity_y_delta',

        # 添加对手交互特征
        'nearest_opp_dist', 'closing_speed', 'num_nearby_opp_3', 'num_nearby_opp_5',
        'mirror_wr_vx', 'mirror_wr_vy', 'mirror_offset_x', 'mirror_offset_y', 'mirror_wr_dist',
        'pressure', 'under_pressure', 'pressure_x_speed',
        'mirror_similarity', 'mirror_offset_dist', 'mirror_alignment'

        # 'velocity_x_roll3', 'velocity_x_roll6', 'velocity_x_roll9', 'velocity_x_std3', 'velocity_x_std6', 'velocity_x_std9'
        # 'velocity_y_roll3', 'velocity_y_roll6', 'velocity_y_roll9', 'velocity_y_std3', 'velocity_y_std6', 'velocity_y_std9',
        # 's_roll3', 's_roll6', 's_roll9', 's_std3', 's_std6', 's_std9'
        # 'a_roll3', 'a_roll6', 'a_roll9', 'a_std3', 'a_std6', 'a_std9'
        # 'qb_distance', 'vel_to_qb_alignment', 'vel_to_qb_perp',
        # 'bearing_to_qb_signed', 'bearing_to_qb_sin', 'bearing_to_qb_cos'
                # 急动特征
        # 'jerk', 'jerk_x', 'jerk_y',
        
        # # 落点曲率特征
        # 'bearing_to_land_signed', 'land_lateral_offset', 'curvature_signed', 'curvature_abs',
        # 'curv_signed_roll3', 'curv_abs_roll3', 'curv_signed_roll5', 'curv_abs_roll5',
    ]
    
    # 合并所有特征列，只返回数据集中存在的
    all_feature_cols = base_feature_cols
    return [c for c in all_feature_cols if c in df.columns]


# ============================================================================
# MAIN SEQUENCE PREPARATION FUNCTION
# ============================================================================
# ============================================================================
# 方向统一工具函数（与训练一致）
# ============================================================================

YARDS_TO_METERS = 0.9144
FPS = 10.0 
FIELD_LENGTH, FIELD_WIDTH = 120.0, 53.3

def wrap_angle_deg(s):
    # map to (-180, 180]
    return ((s + 180.0) % 360.0) - 180.0

def unify_left_direction(df: pd.DataFrame) -> pd.DataFrame:
    """Mirror rightward plays so all samples are 'left' oriented (x,y, dir, o, ball_land)."""
    if 'play_direction' not in df.columns:
        return df
    df = df.copy()
    right = df['play_direction'].eq('right')
    # positions
    if 'x' in df.columns: df.loc[right, 'x'] = FIELD_LENGTH - df.loc[right, 'x']
    if 'y' in df.columns: df.loc[right, 'y'] = FIELD_WIDTH  - df.loc[right, 'y']
    # angles in degrees
    for col in ('dir','o'):
        if col in df.columns:
            df.loc[right, col] = (df.loc[right, col] + 180.0) % 360.0
    # ball landing
    if 'ball_land_x' in df.columns:
        df.loc[right, 'ball_land_x'] = FIELD_LENGTH - df.loc[right, 'ball_land_x']
    if 'ball_land_y' in df.columns:
        df.loc[right, 'ball_land_y'] = FIELD_WIDTH  - df.loc[right, 'ball_land_y']
    return df

def build_play_direction_map(df_in: pd.DataFrame) -> pd.Series:
    """
    Return a Series indexed by (game_id, play_id) with values 'left'/'right'.
    """
    s = (
        df_in[['game_id','play_id','play_direction']]
        .drop_duplicates()
        .set_index(['game_id','play_id'])['play_direction']
    )
    return s

def apply_direction_to_df(df: pd.DataFrame, dir_map: pd.Series) -> pd.DataFrame:
    """
    Attach play_direction (if missing) and then unify to 'left'.
    """
    if 'play_direction' not in df.columns:
        dir_df = dir_map.reset_index()  # -> columns: game_id, play_id, play_direction
        df = df.merge(dir_df, on=['game_id','play_id'], how='left', validate='many_to_one')
    return unify_left_direction(df)

def invert_to_original_direction(x_u, y_u, play_dir_right: bool):
    """Invert unified (left) coordinates back to original play direction."""
    if not play_dir_right:
        return float(x_u), float(y_u)
    return float(FIELD_LENGTH - x_u), float(FIELD_WIDTH - y_u)

# ============================================================================
# 修改后的 MAIN SEQUENCE PREPARATION FUNCTION（包含方向统一）
# ============================================================================

def prepare_sequences_fixed(input_df, output_df=None, test_template=None, is_training=True, window_size=Config.WINDOW_SIZE):
    """
    准备序列数据的主函数（包含方向统一和所有特征工程）
    """
    # ===== 1. 方向统一处理 =====
    print("应用方向统一...")
    dir_map = build_play_direction_map(input_df)
    
    # 统一输入数据方向
    input_df_u = apply_direction_to_df(input_df, dir_map)
    
    if is_training:
        # 统一输出数据方向
        out_u = apply_direction_to_df(output_df, dir_map)
        target_rows = out_u
        target_groups = out_u[['game_id','play_id','nfl_id']].drop_duplicates()
    else:
        # 确保测试模板有play_direction
        if 'play_direction' not in test_template.columns:
            dir_df = dir_map.reset_index()
            test_template = test_template.merge(dir_df, on=['game_id','play_id'], how='left', validate='many_to_one')
        target_rows = test_template
        target_groups = target_rows[['game_id','play_id','nfl_id']].drop_duplicates()
    
    # 验证方向统一是否成功
    assert target_rows[['game_id','play_id','play_direction']].isna().sum().sum() == 0, \
        "play_direction merge failed; check (game_id, play_id) coverage"
    print("方向统一完成:", target_rows['play_direction'].value_counts(dropna=False).to_dict())
    
    # ===== 2. 特征工程 =====
    print("开始特征工程...")
    input_df_u = create_base_features(input_df_u)
    input_df_u = create_lag_features(input_df_u, window_size)
    opponent_features = get_opponent_features(input_df_u)
    input_df_u = input_df_u.merge(opponent_features, on=['game_id', 'play_id', 'nfl_id'], how='left')
    input_df_u = add_advanced_features(input_df_u)
    
    # 获取特征列
    feature_cols = get_feature_columns(input_df_u)
    print(f"使用 {len(feature_cols)} 个特征" + (" (训练)" if is_training else ""))
    
    # 设置索引用于分组操作
    input_df_u.set_index(['game_id', 'play_id', 'nfl_id'], inplace=True)
    grouped = input_df_u.groupby(level=['game_id', 'play_id', 'nfl_id'])
    
    if is_training:
        sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids = [], [], [], [], []
        
        for _, row in tqdm(target_groups.iterrows(), desc="处理训练序列"):
            key = (row['game_id'], row['play_id'], row['nfl_id'])
            try:
                group_df = grouped.get_group(key)
            except KeyError:
                continue
            
            # 创建输入窗口
            input_window = group_df.tail(window_size)
            if len(input_window) < window_size:
                continue  # 训练时跳过长度不足的序列
            
            # 填充缺失值
            input_window = input_window.fillna(group_df.mean(numeric_only=True))
            seq = input_window[feature_cols].values
            
            # 处理NaN值
            if np.isnan(seq).any():
                continue  # 训练时跳过有NaN的序列
            
            sequences.append(seq)
            
            # 获取对应的目标值（注意：目标值已经是统一方向后的）
            out_grp = target_rows[
                (target_rows['game_id'] == row['game_id']) &
                (target_rows['play_id'] == row['play_id']) &
                (target_rows['nfl_id'] == row['nfl_id'])
            ].sort_values('frame_id')
            
            last_x = input_window.iloc[-1]['x']
            last_y = input_window.iloc[-1]['y']
            dx = out_grp['x'].values - last_x
            dy = out_grp['y'].values - last_y
            targets_dx.append(dx)
            targets_dy.append(dy)
            targets_frame_ids.append(out_grp['frame_id'].values)
            
            sequence_ids.append({
                'game_id': key[0],
                'play_id': key[1],
                'nfl_id': key[2],
                'frame_id': input_window.iloc[-1]['frame_id'],
                'play_direction': input_window.iloc[-1]['play_direction']  # 保存方向信息用于推理时还原
            })
        
        return sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids
    
    else:
        # 推理模式
        sequences, sequence_ids = [], []
        
        for _, row in tqdm(target_groups.iterrows(), desc="处理推理序列"):
            key = (row['game_id'], row['play_id'], row['nfl_id'])
            try:
                group_df = grouped.get_group(key)
            except KeyError:
                continue
            
            # 创建输入窗口
            input_window = group_df.tail(window_size)
            if len(input_window) < window_size:
                pad_len = window_size - len(input_window)
                pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
                input_window = pd.concat([pad_df, input_window], ignore_index=True)
            
            # 填充缺失值
            input_window = input_window.fillna(group_df.mean(numeric_only=True))
            seq = input_window[feature_cols].values
            
            # 处理NaN值
            if np.isnan(seq).any():
                seq = np.nan_to_num(seq, nan=0.0)
            
            sequences.append(seq)
            sequence_ids.append({
                'game_id': key[0],
                'play_id': key[1],
                'nfl_id': key[2],
                'frame_id': input_window.iloc[-1]['frame_id'],
                'play_direction': input_window.iloc[-1]['play_direction'],  # 保存方向信息
                'last_x': input_window.iloc[-1]['x'],  # 保存最后位置用于推理时还原
                'last_y': input_window.iloc[-1]['y']
            })
        
        return sequences, sequence_ids

In [ ]:
# ============================================================================
# MODEL（与训练一致，定义在函数外但代码完整）
# ============================================================================

class ImprovedSeqModel(nn.Module):
    def __init__(self, input_dim, horizon):
        super().__init__()
        self.horizon = horizon
        self.gru = nn.GRU(input_dim, 128, num_layers=2, batch_first=True, dropout=0.1)
        self.pool_ln = nn.LayerNorm(128)
        self.pool_attn = nn.MultiheadAttention(128, num_heads=4, batch_first=True)
        self.pool_query = nn.Parameter(torch.randn(1, 1, 128))
        self.head = nn.Sequential(
            nn.Linear(128, 128),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(128, horizon)
        )
    
    def forward(self, x):
        h, _ = self.gru(x)
        B = h.size(0)
        q = self.pool_query.expand(B, -1, -1)
        h_norm = self.pool_ln(h)
        ctx, _ = self.pool_attn(q, h_norm, h_norm)
        ctx = ctx.squeeze(1)
        out = self.head(ctx)
        out = torch.cumsum(out, dim=1)
        return out

# ============================================================================
# INFERENCE 核心：predict函数（独立运行，不依赖外部状态）
# ============================================================================

# ============================================================================
# INFERENCE 核心：修改后的predict函数（包含方向还原）
# ============================================================================

def predict(test: pl.DataFrame, test_input: pl.DataFrame) -> pl.DataFrame:
    # ===== 1. 准备数据 =====
    test_pd = test.to_pandas()
    test_input_pd = test_input.to_pandas()
    
    # ===== 2. 特征工程（包含方向统一）=====
    sequences, sequence_ids = prepare_sequences_fixed(
        test_input_pd, test_template=test_pd, is_training=False, window_size=Config.WINDOW_SIZE
    )
    
    if not sequences:
        return pl.DataFrame({"x": [], "y": []})
    
    X_test = np.array(sequences, dtype=object)
    
    # 保存统一方向后的最后位置
    x_last_u = np.array([s[-1, 0] for s in X_test])  # 统一方向后的x
    y_last_u = np.array([s[-1, 1] for s in X_test])  # 统一方向后的y
    
    # ===== 3. 加载模型 & Scaler =====
    MODEL_DIR = Path("/kaggle/input/1101adddirec/pytorch/default/1/adddirec")  # 根据实际路径修改
    
    model_x_paths = [
        MODEL_DIR / f"model_x_fold{i+1}.pth" for i in range(Config.N_FOLDS)
    ]
    model_y_paths = [
        MODEL_DIR / f"model_y_fold{i+1}.pth" for i in range(Config.N_FOLDS)
    ]
    scaler_paths = [
        MODEL_DIR / f"scaler_fold{i+1}.pkl" for i in range(Config.N_FOLDS)
    ]
    
    # 检查路径是否存在
    for p in model_x_paths + model_y_paths + scaler_paths:
        if not p.exists():
            raise FileNotFoundError(f"请上传模型文件，缺失: {p.name}")

    models_x = []
    models_y = []
    scalers = []
    
    for i in range(Config.N_FOLDS):
        # 模型X
        model_x = ImprovedSeqModel(input_dim=X_test[0].shape[1], horizon=Config.MAX_FUTURE_HORIZON).to(Config.DEVICE)
        model_x.load_state_dict(torch.load(model_x_paths[i], map_location=Config.DEVICE))
        model_x.eval()
        models_x.append(model_x)
        
        # 模型Y
        model_y = ImprovedSeqModel(input_dim=X_test[0].shape[1], horizon=Config.MAX_FUTURE_HORIZON).to(Config.DEVICE)
        model_y.load_state_dict(torch.load(model_y_paths[i], map_location=Config.DEVICE))
        model_y.eval()
        models_y.append(model_y)
        
        # Scaler
        scaler = joblib.load(scaler_paths[i])
        scalers.append(scaler)
    
    # ===== 4. 预测（在统一方向坐标系中） =====
    all_dx, all_dy = [], []
    for mx, my, sc in zip(models_x, models_y, scalers):
        X_scaled = np.stack([sc.transform(s) for s in X_test])
        X_tensor = torch.tensor(X_scaled.astype(np.float32)).to(Config.DEVICE)
        
        mx.eval()
        my.eval()
        with torch.no_grad():
            dx = mx(X_tensor).cpu().numpy()  # 统一方向坐标系中的dx
            dy = my(X_tensor).cpu().numpy()  # 统一方向坐标系中的dy
        all_dx.append(dx)
        all_dy.append(dy)
    
    ens_dx = np.mean(all_dx, axis=0)
    ens_dy = np.mean(all_dy, axis=0)
    
    # ===== 5. 方向还原 + 构造提交结果 =====
    rows = []
    H = ens_dx.shape[1]  # 预测步数
    
    for i, sid in enumerate(sequence_ids):
        # 获取该序列对应的所有frame_id
        fids = test_pd[
            (test_pd['game_id'] == sid['game_id']) &
            (test_pd['play_id'] == sid['play_id']) &
            (test_pd['nfl_id'] == sid['nfl_id'])
        ]['frame_id'].sort_values().tolist()
        
        play_dir_right = (sid['play_direction'] == 'right')
        
        for t, fid in enumerate(fids):
            tt = min(t, H - 1)
            
            # 在统一方向坐标系中的绝对位置
            x_u = np.clip(x_last_u[i] + ens_dx[i, tt], 0, 120)
            y_u = np.clip(y_last_u[i] + ens_dy[i, tt], 0, 53.3)
            
            # 转换回原始方向坐标系
            x_orig, y_orig = invert_to_original_direction(x_u, y_u, play_dir_right)
            
            # 确保在场地范围内
            x_orig = np.clip(x_orig, 0, 120)
            y_orig = np.clip(y_orig, 0, 53.3)
            
            rows.append({
                'x': float(x_orig),
                'y': float(y_orig)
            })
    
    return pl.DataFrame(rows)

# ============================================================================
# Kaggle 推理服务器（保持不变）
# ============================================================================

# 确保导入官方评估库
try:
    from kaggle_evaluation.nfl_inference_server import NFLInferenceServer
except ImportError as e:
    raise ImportError("请确保使用 Kaggle 的 NFL Inference 评测内核")

# 创建推理服务器并运行
inference_server = NFLInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/nfl-big-data-bowl-2026-prediction/',))